Make the tissue and cell type predictor atlas

In [2]:
import mysql.connector
import pandas as pd
import numpy as np

In [3]:
conn = mysql.connector.connect(user='root', password='password', host='127.0.0.1', port='3306',
                               database='expression_atlas2', auth_plugin='mysql_native_password')
mycursor = conn.cursor()

# check the connection
if conn.is_connected():
    print("connection succesfull")
else:
    print("no connection")


connection succesfull


In [4]:
proteinData = pd.read_csv('../Atlas_and_parser/Atlas_output/proteome_nsaf_update0806.csv')
proteinData.head(25)

,assay_id,uniprot_id,NSAF
0,13988,A2RRP1,0.000259
1,13988,A5YM72,0.000372
2,13988,O00483,0.007590
3,13988,O14495,0.000988
4,13988,O14594,0.000931
5,13988,O14926,0.000625
6,13988,O15020,0.009003
7,13988,O15230,0.001997
8,13988,O60242,0.000202
9,13988,O60423,0.000236


In [5]:
proteinData.shape

(4850267, 3)

Get tissue data from database which will serve as labels for the data to build the classifier
First start with only healthy tissues

In [6]:
tissuesql = """SELECT tissue_to_assay.assay_id, tissue.cell_type, tissue.tissue_name, tissue.disease_status
             FROM tissue_to_assay JOIN tissue ON tissue_to_assay.tissue_id = tissue.tissue_id"""
tissueData = pd.read_sql_query(tissuesql,conn)
tissueData.head()

,assay_id,cell_type,tissue_name,disease_status
0,13988,Brain,Brain,Healthy
1,13989,Brain,Brain,Healthy
2,13990,Brain,Brain,Healthy
3,13991,Brain,Brain,Healthy
4,13992,Brain,Brain,Healthy


In [7]:
index1 = tissueData[tissueData['tissue_name'] == 'Hela'].index
tissueData.drop(index1, inplace=True)
tissueData_healthy = tissueData.loc[tissueData['disease_status'] == "Healthy"]
tissueData_all = tissueData

In [8]:
tissueData_healthy = tissueData_healthy.drop(['cell_type', 'disease_status'], axis=1)
tissueData_all = tissueData_all.drop(['cell_type', 'disease_status'], axis=1)
tissueData_all.head()

,assay_id,tissue_name
0,13988,Brain
1,13989,Brain
2,13990,Brain
3,13991,Brain
4,13992,Brain


Make a pivot table of assays with the NSAF values for each protein (0 if not detected)

In [9]:
assay_atlas = pd.pivot_table(proteinData, values = 'NSAF', index = 'assay_id', columns = 'uniprot_id').fillna(0).reset_index()
assay_atlas.head(15)
#assay_atlas

uniprot_id,assay_id,A0A024RBG1,A0A075B6H7,A0A075B6H8,A0A075B6H9,A0A075B6I0,A0A075B6I1,A0A075B6I9,A0A075B6J1,A0A075B6J2,...,Q9Y6X8,Q9Y6X9,Q9Y6Y0,Q9Y6Y1,Q9Y6Y8,Q9Y6Y9,Q9Y6Z4,Q9Y6Z5,Q9Y6Z7,W5XKT8
0,13988,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,13989,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,13990,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,13991,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,13992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,13993,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,13994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,13995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,13996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,13997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
atlas_healthy = pd.merge(assay_atlas, tissueData_healthy, on = 'assay_id')
atlas_all = pd.merge(assay_atlas, tissueData_all, on = 'assay_id')
print(atlas_healthy.shape)
print(atlas_all.shape)
#atlas

(3342, 11677)
(8748, 11677)


In [11]:
atlas_healthy.head(15)

,assay_id,A0A024RBG1,A0A075B6H7,A0A075B6H8,A0A075B6H9,A0A075B6I0,A0A075B6I1,A0A075B6I9,A0A075B6J1,A0A075B6J2,...,Q9Y6X9,Q9Y6Y0,Q9Y6Y1,Q9Y6Y8,Q9Y6Y9,Q9Y6Z4,Q9Y6Z5,Q9Y6Z7,W5XKT8,tissue_name
0,13988,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Brain
1,13989,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Brain
2,13990,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Brain
3,13991,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Brain
4,13992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Brain
5,13993,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Brain
6,13994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Brain
7,13995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Brain
8,13996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Brain
9,13997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Brain


Remove assay ids (unneeded feature for building the classifier)

In [12]:
assays_healthy = atlas_healthy.pop('assay_id')
assays_all = atlas_all.pop('assay_id')

In [13]:
with pd.option_context('display.max_rows', None,'display.max_columns', None ):
    print(atlas_healthy['tissue_name'].unique())
with pd.option_context('display.max_rows', None,'display.max_columns', None ):
    print(atlas_all['tissue_name'].unique())

['Brain' 'Blood' 'Adrenal gland' 'B-cells' 'T-cells' 'Colon' 'Esophagus'
 'Gall bladder' 'Small intestine' 'Heart' 'Kidney' 'Liver' 'Lung'
 'Monocytes' 'NK-cells' 'Ovary' 'Pancreas' 'Placenta' 'Prostate' 'Rectum'
 'Retina' 'Spinal cord' 'Testis' 'Urinary bladder' 'Dental plaque' 'PBMC'
 'Sputum' 'Cartilage' 'Ureter' 'Bone' 'Skin' 'Follicular fluid'
 'Peritoneal dialysis fluid' 'Menstrual fluid' 'Eye' 'Cerebrospinal fluid'
 'Nasal polyps' 'Tonsil' 'Parotid gland' 'Cervix' 'Skeletal muscle'
 'Tooth' 'Adipose tissue' 'Umbilical cord' 'Urine' 'Appendix'
 'Bone marrow' 'Duodenum' 'Endometrium' 'fallopian tube/oviduct'
 'lymph node' 'Pituitary gland' 'Salivary gland' 'Smooth muscle' 'Spleen'
 'Stomach' 'Thyroid' 'Anus' 'Breast' 'Epidymis' 'Paratyroid gland'
 'Seminal vesicle' 'Trachea' 'Uterus']
['Brain' 'Blood' 'Colon' 'Liver' 'Adrenal gland' 'B-cells' 'T-cells'
 'Esophagus' 'Gall bladder' 'Small intestine' 'Heart' 'Kidney' 'Lung'
 'Monocytes' 'NK-cells' 'Ovary' 'Pancreas' 'Placenta' 'Prost

In [14]:
t = set(atlas_all['tissue_name'].unique()) -set(atlas_healthy['tissue_name'].unique())
print('the following tissues are not present in the healthy tissue predictor {}'.format(t))

the following tissues are not present in the healthy tissue predictor {'Synovial tissue', 'Lymph node', 'Bronchoalveolar lavage', 'Unknown'}


In [17]:
atlas_healthy.to_csv('../Atlas_and_parser/Atlas_output/tissue_predictor_healthy_atlas_update0806.csv', index=False)

In [18]:
atlas_all.to_csv('../Atlas_and_parser/Atlas_output/tissue_predictor_all_disease_atlas_update0806.csv', index=False)